# Cracking Java's Truncated LCG
Many security articles warn developers about using non-cryptographically secure pseudorandom number generators, but they never seem to give any concrete examples. Here's a nontrivial example of recovering the internal state of Java's `util.Random`. Note, this should work for _any_ language that uses Java's standard library (Scala, Groovy, Clojure, Kotlin, etc). In this example, we get 64 bytes (16 unsigned integers) from Java Random's `nextBytes` function.

In [1]:
from samson.prngs.lcg import LCG
from samson.utilities.bytes import Bytes

In [2]:
# Scala code to generate random bytes
# object Main extends App{
    
#     def convertBytesToHex(bytes: Seq[Byte]): String = {
#         val sb = new StringBuilder
#         for (b <- bytes) {
#             sb.append(String.format("%02x", Byte.box(b)))
#         }
#         sb.toString
#     }

#     val bytes = new Array[Byte](64)
#     scala.util.Random.nextBytes(bytes)
#     println(convertBytesToHex(bytes))
# }

java_output = Bytes(0x8a2acb2ada63472ff089833623a91b5e8fd19890c2c89a563f9ae3f58fc5b6a8d7bf0455b93137175dfd6d98d838ae6fd4f24fabe4afba15769ea09b23057ce0)

First, we'll need to unpack the bytes into the original integer outputs so we can find their mathematical correlation. We'll also need to peruse the Java source code to find the type of PRNG and its parameters. Java (OpenJDK at least) uses a truncated linear congruential generator (TLCG) (see the [PRNG table](https://github.com/wildcardcorp/samson/blob/master/doc/prng_table.md) in the documentation). The problem with this type of PRNG is that the output is a truncation of the internal state. In this case, we're missing 16 bits per output. Fortunately, samson includes a special function to deal with these. Using some LLL magic and a bit of bruteforce, we can still recover the internal state.

We'll chunk our integer outputs into three parts: `analysis_outputs` for which we'll find the correlation between, `reference_outputs` for which the algorithm will compare a candidate TLCG against, and `next_outputs` for the demonstration in the last cell.

In [7]:
# Unpack bytes into integers. Java packs them in 'backwards' ([4, 3, 2, 1, 4, 3, 2, 1])
# http://hg.openjdk.java.net/jdk8/jdk8/jdk/file/687fd7c7986d/src/share/classes/java/util/Random.java#l227
integer_outputs = [chunk[::-1].int() for chunk in java_output.chunk(4)]
analysis_outputs, reference_outputs, next_outputs = integer_outputs[:8], integer_outputs[8:12], integer_outputs[-4:]

# LCG parameters from Java source code
# http://hg.openjdk.java.net/jdk8/jdk8/jdk/file/687fd7c7986d/src/share/classes/java/util/Random.java#l88
cracked_lcg = LCG.crack_truncated(analysis_outputs, reference_outputs, multiplier=0x5DEECE66D, increment=0xB, modulus=2**48, trunc_amount=16)

Seedspace searched:  52%|█████▏    | 34006/65536 [00:00<00:00, 49910.63seeds/s]


In [8]:
print('Cracked LCG', cracked_lcg)
print('Next four LCG outputs', [cracked_lcg.generate() for _ in range(4)])
print('Next actual outputs  ', next_outputs)

Cracked LCG <LCG: X=122794068710207, a=25214903917, c=140169650869055, m=281474976710656, trunc=16>
Next four LCG outputs [2874143444, 364556260, 2610994806, 3766224163]
Next actual outputs   [2874143444, 364556260, 2610994806, 3766224163]
